In [ ]:
!pip install requests

In [ ]:
import os
import json
import requests

In [ ]:
# NER
MODEL_NAME = "dslim/bert-base-NER"
API_URL = f"https://api-inference.huggingface.co/models/{MODEL_NAME}"
API_TOKEN = "your_huggingface_token_here"  # Replace with your actual token
headers = {
    "Authorization": f"Bearer {API_TOKEN}",
}

def query(payload):
      response = requests.post(API_URL, headers=headers, json=payload)
      return response.json()

output = query({
  "inputs": "My name is John;  but you can call me ",
})
print(output)

[{'entity_group': 'PER', 'score': 0.9954715, 'word': 'John', 'start': 11, 'end': 15}]


In [ ]:
# QA
MODEL_NAME = "deepset/roberta-base-squad2"
API_URL = f"https://api-inference.huggingface.co/models/{MODEL_NAME}"
API_TOKEN = "your_huggingface_token_here"  # Replace with your actual token
headers = {
    "Authorization": f"Bearer {API_TOKEN}",
}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

payload = {
    "inputs": {
        "question": "Who founded Apple Inc.?",
        "context": (
            "The company is headquartered in Cupertino, California."
            "Apple Inc. was founded by Steve Jobs and Steve Wozniak in 1976."
        )
    }
}

output = query(payload)
print(output)

{'score': 0.9603421092033386, 'start': 80, 'end': 108, 'answer': 'Steve Jobs and Steve Wozniak'}


In [ ]:
# Auto-QA Squad
import os
import time
import requests
import pandas as pd
from datasets import load_dataset

# --------------------------
# Configuration
# --------------------------
MODEL_NAME = "deepset/roberta-base-squad2"
API_URL = f"https://api-inference.huggingface.co/models/{MODEL_NAME}"
API_TOKEN = os.environ.get("HF_TOKEN", "your_huggingface_token_here")  # Replace with your actual token
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

NUM_EXAMPLES = 100     # how many QA pairs to process
SPLIT = "validation"   # "train" or "validation"
OUTPUT_CSV = "squad_v2_qa_answers.csv"
OUTPUT_JSON = "squad_v2_qa_answers.json"

# --------------------------
# API call helper with basic retries
# --------------------------
def query_qa(question: str, context: str, max_retries: int = 5, sleep_seconds: float = 2.0):
    payload = {
        "inputs": {
            "question": question,
            "context": context
        }
    }
    for attempt in range(max_retries):
        resp = requests.post(API_URL, headers=HEADERS, json=payload)
        status = resp.status_code
        text = resp.text
        # Handle cold starts (503) or transient server issues
        if status == 503:
            time.sleep(sleep_seconds)
            continue
        try:
            data = resp.json()
        except Exception:
            # Non-JSON (e.g., HTML error page)
            return {"error": f"Non-JSON response (status={status})", "raw": text}
        # If model returns an error key, optionally wait for model
        if isinstance(data, dict) and "error" in data:
            # If the server suggests waiting time for cold start, sleep then retry
            if "estimated_time" in data:
                time.sleep(min(float(data["estimated_time"]) + 0.5, 10))
                continue
            # Otherwise return the error
            return data
        return data
    return {"error": f"Failed after {max_retries} retries"}

# --------------------------
# Load SQuAD v2 data
# --------------------------
# SQuAD v2 contains context, question, answers; validation split has 11,873 examples[1][2].
dataset = load_dataset("rajpurkar/squad_v2", split=SPLIT)  # requires `datasets` package[1][2][9]

# Subset to first NUM_EXAMPLES question-context pairs
subset = dataset.select(range(min(NUM_EXAMPLES, len(dataset))))

# --------------------------
# Iterate and collect answers
# --------------------------
rows = []
for i, ex in enumerate(subset):
    q = ex["question"]
    ctx = ex["context"]
    qa_result = query_qa(q, ctx)

    # deepset/roberta-base-squad2 via Inference API typically returns dict with answer/score/start/end
    if isinstance(qa_result, dict) and "answer" in qa_result:
        answer = qa_result.get("answer", "")
        score = qa_result.get("score", None)
        start = qa_result.get("start", None)
        end = qa_result.get("end", None)
        error = None
    else:
        # capture errors or unexpected shapes
        answer = ""
        score = None
        start = None
        end = None
        error = qa_result if isinstance(qa_result, dict) else {"unexpected": str(qa_result)}

    print(f"[{i+1}/{len(subset)}] Q: {q}\nA: {answer}\n")
    rows.append({
        "id": ex.get("id", ""),
        "title": ex.get("title", ""),
        "question": q,
        "context": ctx,
        "gold_answers": ex.get("answers", {}),
        "pred_answer": answer,
        "pred_score": score,
        "pred_start": start,
        "pred_end": end,
        "error": error
    })

# --------------------------
# Save results
# --------------------------
df = pd.DataFrame(rows)
df.to_csv(OUTPUT_CSV, index=False)
df.to_json(OUTPUT_JSON, orient="records", force_ascii=False, indent=2)

print(f"\nSaved results to {OUTPUT_CSV} and {OUTPUT_JSON}")

[1/100] Q: In what country is Normandy located?
A: France

[2/100] Q: When were the Normans in Normandy?
A: 10th and 11th centuries

[3/100] Q: From which countries did the Norse originate?
A: Denmark, Iceland and Norway

[4/100] Q: Who was the Norse leader?
A: Rollo

[5/100] Q: What century did the Normans first gain their separate identity?
A: 10th

[6/100] Q: Who gave their name to Normandy in the 1000's and 1100's
A: The Normans

[7/100] Q: What is France a region of?
A: Normandy

[8/100] Q: Who did King Charles III swear fealty to?
A: King Charles III of West Francia

[9/100] Q: When did the Frankish identity emerge?
A: first half of the 10th century

[10/100] Q: Who was the duke in the battle of Hastings?
A: William the Conqueror

[11/100] Q: Who ruled the duchy of Normandy
A: Richard I of Normandy

[12/100] Q: What religion were the Normans
A: Christian

[13/100] Q: What type of major impact did the Norman dynasty have on modern Europe?
A: political, cultural and military

[14/1